In [1]:
import requests
import random
import json
import random
import webbrowser

In [2]:
registryBaseUrl = "https://sunbirdrc-sandbox.xiv.in/registry/"
registryUrl = "https://sunbirdrc-sandbox.xiv.in/registry/api/v1/"
keycloakUrl = "https://sunbirdrc-sandbox.xiv.in/"
client_id='sandbox-client-1'
client_secret='968e20a9-7500-465c-9117-f2d234f6a378'

In [3]:
response = requests.get('%s/health'%(registryBaseUrl))
print(response.text)
assert response.status_code == 200 

{"id":"sunbird-rc.registry.health","ver":"1.0","ets":1663312622824,"params":{"resmsgid":"","msgid":"3487f3be-0418-4f24-946a-304eaa394518","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"name":"sunbirdrc-registry-api","healthy":false,"checks":[{"name":"sunbirdrc.database","healthy":true,"err":"","errmsg":""},{"name":"sunbird.signature.service","healthy":false,"err":"","errmsg":""}]}}


# get client token


In [97]:
def getToken(clientId, clientSecret):
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }

    data = {
      'client_id': clientId,
      'client_secret': clientSecret,
      'grant_type': 'client_credentials'
    }

    response = requests.post('%sauth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloakUrl), headers=headers, data=data)
    print(response.json())
#     print(response.status_code)
    token = response.json()["access_token"]
    return token

token = getToken(client_id, client_secret)
print(token)

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJNWjA2UEVqTWlfNW5uejE5ZEprZmtDVTFDQkM5elpVYVhuM2NYemUxVjgwIn0.eyJleHAiOjE2NjMzMjM5NDEsImlhdCI6MTY2MzMyMzM0MSwianRpIjoiODExZmU3NWQtZWE5Mi00ZTVmLThhMWQtOTQzMDcwMDJmODg4IiwiaXNzIjoiaHR0cHM6Ly9zdW5iaXJkcmMtc2FuZGJveC54aXYuaW4vYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0MWUzYzk5Ni05MjE5LTRkMjYtYTc4OS0xM2VkMGM5NjcwZTYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzYW5kYm94LWNsaWVudC0xIiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJwcm9maWxlIGVtYWlsIiwiY2xpZW50SWQiOiJzYW5kYm94LWNsaWVudC0xIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJjbGllbnRIb3N0IjoiMTA2LjUxLjM3LjEwNiIsInByZWZlcnJlZF91c2VybmFtZSI6InNlcnZpY2UtYWNjb3VudC1zYW5kYm94LWNsaWVudC0xIiwiY2xpZW50QWRkcmVzcyI6IjEwNi41MS4zNy4xMDYifQ.ik0XjlcETtIYLSU

# create a schema

In [38]:

schemaRequestBody = {
  "name": "TrainingCertificateV"+str(random.randrange(0, 100,1)),
    "schema": "{\n  \"$schema\": \"http://json-schema.org/draft-07/schema\",\n  \"type\": \"object\",\n  \"properties\": {\n    \"TrainingCertificate\": {\n      \"$ref\": \"#/definitions/TrainingCertificate\"\n    }\n  },\n  \"required\": [\n    \"TrainingCertificate\"\n  ],\n  \"title\": \"TrainingCertificate\",\n  \"definitions\": {\n    \"TrainingCertificate\": {\n      \"$id\": \"#/properties/TrainingCertificate\",\n      \"type\": \"object\",\n      \"title\": \"The TrainingCertificate Schema\",\n      \"required\": [\n        \"name\",\n        \"contact\"\n      ],\n      \"properties\": {\n        \"name\": {\n          \"type\": \"string\"\n        },\n        \"trainingTitle\": {\n          \"type\": \"string\"\n        },\n        \"contact\": {\n          \"type\": \"string\"\n        },\n        \"date\": {\n          \"type\": \"string\",\n          \"format\": \"date\"\n        },\n        \"note\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  \"_osConfig\": {\n    \"uniqueIndexFields\": [\n      \"contact\"\n    ],\n    \"ownershipAttributes\": [],\n    \"roles\": [],\n    \"inviteRoles\": [\n      \"anonymous\"\n    ],\n    \"enableLogin\": false,\n    \"credentialTemplate\": {\n      \"@context\": [\n        \"https://www.w3.org/2018/credentials/v1\",\n        \"https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill\"\n      ],\n      \"type\": [\n        \"VerifiableCredential\"\n      ],\n      \"issuanceDate\": \"2021-08-27T10:57:57.237Z\",\n      \"credentialSubject\": {\n        \"type\": \"Person\",\n        \"name\": \"{{name}}\",\n        \"trainedOn\": \"{{trainingTitle}}\"\n      },\n      \"issuer\": \"did:web:sunbirdrc.dev/vc/skill\"\n    },\n    \"certificateTemplates\": {\n      \"html\": \"https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html\",\n      \"svg\": \"https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.svg\"\n    }\n  }\n}"
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("POST", registryUrl + "Schema", headers=headers, json=schemaRequestBody)

print(response.status_code)
print(response.text)
assert response.status_code == 200 
schemaOsid = response.json()["result"]["Schema"]["osid"]
print(schemaOsid)

200
{"id":"sunbird-rc.registry.create","ver":"1.0","ets":1663313445968,"params":{"resmsgid":"","msgid":"caa8e4ea-6795-4d81-be20-4e8f9cd26524","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Schema":{"osid":"1-83c677c2-02b8-4bad-850c-376cae4f7024"}}}
1-83c677c2-02b8-4bad-850c-376cae4f7024


In [39]:
response

<Response [200]>

# get all schemas

In [40]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("POST", registryUrl + "Schema/search", headers=headers, json=schemaRequestBody)

print(response.json())
for j in response.json():
    print(j['name'])

[{'name': 'TrainingCertificateV28', 'schema': '{\n  "$schema": "http://json-schema.org/draft-07/schema",\n  "type": "object",\n  "properties": {\n    "TrainingCertificate": {\n      "$ref": "#/definitions/TrainingCertificate"\n    }\n  },\n  "required": [\n    "TrainingCertificate"\n  ],\n  "title": "TrainingCertificate",\n  "definitions": {\n    "TrainingCertificate": {\n      "$id": "#/properties/TrainingCertificate",\n      "type": "object",\n      "title": "The TrainingCertificate Schema",\n      "required": [\n        "name",\n        "contact"\n      ],\n      "properties": {\n        "name": {\n          "type": "string"\n        },\n        "trainingTitle": {\n          "type": "string"\n        },\n        "contact": {\n          "type": "string"\n        },\n        "date": {\n          "type": "string",\n          "format": "date"\n        },\n        "note": {\n          "type": "string"\n        }\n      }\n    }\n  },\n  "_osConfig": {\n    "uniqueIndexFields": [\n      "

In [41]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("GET", registryUrl + "Schema", headers=headers)
print(response)
print(response.json())
print(len(response.json()))

<Response [200]>
[{'schema': '{\n  "$schema": "http://json-schema.org/draft-07/schema",\n  "type": "object",\n  "properties": {\n    "TrainingCertificate": {\n      "$ref": "#/definitions/TrainingCertificate"\n    }\n  },\n  "required": [\n    "TrainingCertificate"\n  ],\n  "title": "TrainingCertificate",\n  "definitions": {\n    "TrainingCertificate": {\n      "$id": "#/properties/TrainingCertificate",\n      "type": "object",\n      "title": "The TrainingCertificate Schema",\n      "required": [\n        "name",\n        "contact"\n      ],\n      "properties": {\n        "name": {\n          "type": "string"\n        },\n        "trainingTitle": {\n          "type": "string"\n        },\n        "contact": {\n          "type": "string"\n        },\n        "date": {\n          "type": "string",\n          "format": "date"\n        },\n        "note": {\n          "type": "string"\n        }\n      }\n    }\n  },\n  "_osConfig": {\n    "uniqueIndexFields": [\n      "contact"\n    ],\

In [42]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("GET", registryUrl + "Schema", headers=headers)
print(response)
print(response.json())
print(len(response.json()))

<Response [200]>
[{'schema': '{\n  "$schema": "http://json-schema.org/draft-07/schema",\n  "type": "object",\n  "properties": {\n    "TrainingCertificate": {\n      "$ref": "#/definitions/TrainingCertificate"\n    }\n  },\n  "required": [\n    "TrainingCertificate"\n  ],\n  "title": "TrainingCertificate",\n  "definitions": {\n    "TrainingCertificate": {\n      "$id": "#/properties/TrainingCertificate",\n      "type": "object",\n      "title": "The TrainingCertificate Schema",\n      "required": [\n        "name",\n        "contact"\n      ],\n      "properties": {\n        "name": {\n          "type": "string"\n        },\n        "trainingTitle": {\n          "type": "string"\n        },\n        "contact": {\n          "type": "string"\n        },\n        "date": {\n          "type": "string",\n          "format": "date"\n        },\n        "note": {\n          "type": "string"\n        }\n      }\n    }\n  },\n  "_osConfig": {\n    "uniqueIndexFields": [\n      "contact"\n    ],\

# Tests for newly added schem is here:

In [43]:

resp = requests.get("%sapi/docs/swagger.json"%registryBaseUrl)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrls = [f for f in swaggerJson["paths"].keys() if ".json" in f]
print(jsonUrls)
jsonUrl = jsonUrls[0]

resp = requests.get("%s%s"%(registryBaseUrl, jsonUrl))
assert resp.status_code == 200

resp.json()
entities = list(resp.json().keys())
print("Available entities ", entities)
entity_name='TrainingCertificate'
print("Using entity %s"%entity_name)


['/api/docs/TrainingCertificate.json', '/api/docs/Schema.json']
Available entities  ['TrainingCertificate']
Using entity TrainingCertificate


# Upload Credential Template

In [61]:
from io import StringIO
def string_to_file(contents):
    buffer = StringIO()
    buffer.write(contents)
    buffer.seek(0)
    return buffer

In [68]:
# path: deps/credentialTemplate.json
files=[
  ('files',('file',string_to_file('{"@context":["https://www.w3.org/2018/credentials/v1","https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill"],"type":["VerifiableCredential"],"issuanceDate":"2021-08-27T10:57:57.237Z","credentialSubject":{"type":"Person","name":"{{name}}","trainedOn":"{{trainingTitle}}"},"issuer":"did:web:sunbirdrc.dev/vc/skill"}')
            ,'application/octet-stream'))
]
headers = {
    'Authorization': 'Bearer ' + token
}
resp = requests.post("%s%s"%(registryBaseUrl, '/api/v1/Schema/%s/credentialTemplate/documents'%schemaOsid), json={}, headers=headers, files=files)
print(resp)
print(resp.json()['documentLocations'][0])
credentialTemplatePath = "minio://" + resp.json()['documentLocations'][0]

<Response [200]>
Schema/1-83c677c2-02b8-4bad-850c-376cae4f7024/credentialTemplate/documents/dd5a328c-c4a7-4b1d-96ed-e8bdec2acc1c-file


# Upload certificate template

In [69]:
# path: deps/credentialTemplate.json
files=[
  ('files',('file', open('deps/TrainingCertificate.html', 'rb')
            ,'application/octet-stream'))
]
headers = {
    'Authorization': 'Bearer ' + token
}
resp = requests.post("%s%s"%(registryBaseUrl, '/api/v1/Schema/%s/certificateTemplate/documents'%schemaOsid), json={}, headers=headers, files=files)
print(resp)
print(resp.json()['documentLocations'][0])
certificateTemplatePath = "minio://" + resp.json()['documentLocations'][0]

<Response [200]>
Schema/1-83c677c2-02b8-4bad-850c-376cae4f7024/certificateTemplate/documents/cba4b503-0798-4e2e-a82c-2bdb3aaa67c8-file


# Update schema with templates

In [ ]:
schemaRequestBody = {
  "schema": "{\n  \"$schema\": \"http://json-schema.org/draft-07/schema\",\n  \"type\": \"object\",\n  \"properties\": {\n    \"TrainingCertificate\": {\n      \"$ref\": \"#/definitions/TrainingCertificate\"\n    }\n  },\n  \"required\": [\n    \"TrainingCertificate\"\n  ],\n  \"title\": \"TrainingCertificate\",\n  \"definitions\": {\n    \"TrainingCertificate\": {\n      \"$id\": \"#/properties/TrainingCertificate\",\n      \"type\": \"object\",\n      \"title\": \"The TrainingCertificate Schema\",\n      \"required\": [\n        \"name\",\n        \"contact\"\n      ],\n      \"properties\": {\n        \"name\": {\n          \"type\": \"string\"\n        },\n        \"trainingTitle\": {\n          \"type\": \"string\"\n        },\n        \"contact\": {\n          \"type\": \"string\"\n        },\n        \"date\": {\n          \"type\": \"string\",\n          \"format\": \"date\"\n        },\n        \"note\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  \"_osConfig\": {\n    \"uniqueIndexFields\": [\n      \"contact\"\n    ],\n    \"ownershipAttributes\": [],\n    \"roles\": [],\n    \"inviteRoles\": [\n      \"anonymous\"\n    ],\n    \"enableLogin\": false,\n  "
    +"  \"credentialTemplate\": \""+credentialTemplatePath+"\",\n    \"certificateTemplates\": {\n \"html\":\""+certificateTemplatePath+"\"   }\n  }\n}",
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("PUT", registryBaseUrl + "/api/v1/Schema/"+schemaOsid, headers=headers, json=schemaRequestBody)
print(response)
print(response.text)
headers = {
  'Content-Type': 'application/json'
}

# Testing if templates are updated in schema

In [ ]:

headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}
response = requests.request("GET", registryBaseUrl + "/registry/TrainingCertificate", headers=headers)
print(response)
print(json.loads(response.json()["result"]["content"])["definitions"]["TrainingCertificate"])
response.json()
print("createdAt" in json.loads(response.json()["result"]["content"])["definitions"]["TrainingCertificate"]["properties"])
htmlTemplatePath = json.loads(response.json()["result"]["content"])["_osConfig"]["certificateTemplates"]["html"]
credPath = json.loads(response.json()["result"]["content"])["_osConfig"]["credentialTemplate"]
print(htmlTemplatePath)
assert htmlTemplatePath == certificateTemplatePath
assert credPath == credentialTemplatePath

In [92]:

headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}
response = requests.request("GET", registryBaseUrl + "/registry/TrainingCertificate", headers=headers)
print(response)
print(json.loads(response.json()["result"]["content"])["definitions"]["TrainingCertificate"])
response.json()
print("createdAt" in json.loads(response.json()["result"]["content"])["definitions"]["TrainingCertificate"]["properties"])
htmlTemplatePath = json.loads(response.json()["result"]["content"])["_osConfig"]["certificateTemplates"]["html"]
credPath = json.loads(response.json()["result"]["content"])["_osConfig"]["credentialTemplate"]
print(htmlTemplatePath)
assert htmlTemplatePath == certificateTemplatePath
assert credPath == credentialTemplatePath

<Response [200]>
{'$id': '#/properties/TrainingCertificate', 'type': 'object', 'title': 'The TrainingCertificate Schema', 'required': ['name', 'contact'], 'properties': {'name': {'type': 'string'}, 'trainingTitle': {'type': 'string'}, 'contact': {'type': 'string'}, 'date': {'type': 'string', 'format': 'date'}, 'note': {'type': 'string'}}}
False
minio://Schema/1-83c677c2-02b8-4bad-850c-376cae4f7024/certificateTemplate/documents/cba4b503-0798-4e2e-a82c-2bdb3aaa67c8-file


# Create entity

In [98]:
userId =str(random.randint(1e10,1e11))
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}
resp = requests.post("%s%s"%(registryBaseUrl, '/api/v1/%s'%'TrainingCertificate'), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module"
    
}, headers=headers)
print(resp)
assert resp.status_code == 200 or print (resp.text)
idx = resp.json()["result"][entity_name]["osid"]


<Response [200]>


In [99]:
print(resp.json())
resp.status_code, userId
idx = resp.json()["result"][entity_name]["osid"]

{'id': 'open-saber.registry.create', 'ver': '1.0', 'ets': 1663323349905, 'params': {'resmsgid': '', 'msgid': 'd7201ebd-2a74-4151-b68e-f324fc5a2766', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'TrainingCertificate': {'osid': '1-e33b9f90-45af-4b5b-8780-88a0efed0d4c'}}}


# get entity

In [100]:
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token,
    "Accept":"application/vc+ld+json"
}
resp = requests.get("%sapi/v1/%s/%s"%(registryBaseUrl, entity_name, idx), headers=headers)
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "RsaSignature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", "https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill"], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module"}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "RsaSignature2018", "created": "2022-09-16T10:15:50Z", "verificationMethod": "did:india", "proofPurpose": "assertionMethod", "jws": "eyJhbGciOiJQUzI1NiIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..YsMVflpykOUuEMnQRurQ0Tb3cdGw3LgHzDMVkCBM-VJDaiWy2cpOuUlccpdGuK9vf0c995jUb69Xhq6QNyqtPly3no1lSh8oJQsjgmttP---eNWgy4uWZ-ygom9KdpyLrAN8vE6BUhWMd5cchJIfokUsHChEOjFx7_wBOFaEHdNLCFQpINk53WJIocS1Jwyo0VWSk8aeNa9N7eZwBELiN7eGED8BdQ6u2kBdf4zzi821UXT912CQZpAy1-hR_dgOJFlFaqXVnFmX798Kk976DPrgbPiB8hp6gHREr_YzFiyPJGogDxWsub4Aayc8MfKjHdvS6mch98deTJNUo9gYjg"}}


# get certificate pdf

In [101]:
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token,
"Accept":"application/pdf", 
    "template-key": "html"
}
resp = requests.get("%s/api/v1/%s/%s"%(registryBaseUrl, entity_name, idx), headers=headers)
(resp.status_code, resp.content)

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)

%system open 'sample.pdf'

[]

# Delete schema

In [ ]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("DELETE", registryUrl + "Schema/"+schemaOsid, headers=headers)
print(response)
print(response.json())
print(len(response.json()))

# Verify Certificate

In [ ]:
webbrowser.open(keycloakUrl)

In [ ]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("DELETE", registryUrl + "Schema/"+schemaOsid, headers=headers)
print(response)
print(response.json())
print(len(response.json()))

# Verify Certificate

In [ ]:
webbrowser.open(keycloakUrl)

In [24]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("DELETE", registryUrl + "Schema/"+schemaOsid, headers=headers)
print(response)
print(response.json())
print(len(response.json()))

<Response [200]>
{'id': 'sunbird-rc.registry.delete', 'ver': '1.0', 'ets': 1662965760002, 'params': {'resmsgid': '', 'msgid': 'a0e7f014-2d61-4326-935d-f7e7093e45f0', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK'}
5


# Verify Certificate

In [28]:
webbrowser.open(keycloakUrl)

True